In [2]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_compress=True,
    use_ssl=False,
    verify_certs=False
)

# 연결 확인
info = client.info()
print(f"OpenSearch version: {info['version']['number']}")


OpenSearch version: 3.3.0


In [3]:
model_group_body = {
    "name": "reranker_model_group",
    "description": "Cross-encoder models for reranking search results"
}

response = client.transport.perform_request(
    method='POST',
    url='/_plugins/_ml/model_groups/_register',
    body=model_group_body
)
model_group_id = response['model_group_id']
print(f"Model Group ID: {model_group_id}")


Model Group ID: EhbJ_ZsBZOT2qFJ16Sot


In [9]:
register_body = {
    "name": "huggingface/cross-encoders/ms-marco-MiniLM-L-6-v2",
    "version": "1.0.2",
    "model_format": "TORCH_SCRIPT",
    "model_group_id": model_group_id,
}

response = client.transport.perform_request(
    method='POST',
    url='/_plugins/_ml/models/_register',
    body=register_body
)
task_id = response['task_id']
print(f"Registration Task ID: {task_id}")

Registration Task ID: nZQFAJwBOacQ-9pbwDwg


In [10]:
response = client.transport.perform_request(
    method='GET',
    url=f'/_plugins/_ml/tasks/{task_id}',
)

In [ ]:
model_id = response['model_id']
response = client.transport.perform_request(
    method='POST',
    url=f'/_plugins/_ml/models/{model_id}/_deploy',
)

{'model_id': 'n5QFAJwBOacQ-9pbxjwZ', 'task_type': 'REGISTER_MODEL', 'function_name': 'TEXT_SIMILARITY', 'state': 'COMPLETED', 'worker_node': ['J6KE6WNwR-GB48mPvNb_rw'], 'create_time': 1769526902815, 'last_update_time': 1769526923153, 'is_async': True}


In [ ]:
# 상태 확인
status = client.transport.perform_request(
    'GET',
    f'/_plugins/_ml/models/{model_id}'
)

print(f"Model State: {status.get('model_state')}")